## <center> Практическое задание к лекции 6

1. взять любой набор данных для бинарной классификации (можно скачать один из модельных с https://archive.ics.uci.edu/ml/datasets.php)
3. сделать feature engineering
4. обучить любой классификатор (какой вам нравится)

5. далее разделить ваш набор данных на два множества: P (positives) и U (unlabeled). Причем брать нужно не все положительные (класс 1) примеры, а только лишь часть
6. применить random negative sampling для построения классификатора в новых условиях
7. сравнить качество с решением из пункта 4 (построить отчет - таблицу метрик)
8. поэкспериментировать с долей P на шаге 5 (как будет меняться качество модели при уменьшении/увеличении размера P)

Используем [набор данных](https://www.kaggle.com/pritsheta/heart-attack?select=Heart+Attack+Data+Set.csv)

In [1]:
import pandas as pd
import numpy as np

In [2]:
df=pd.read_csv('Heart Attack Data Set.csv')

In [3]:
df

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,57,0,0,140,241,0,1,123,1,0.2,1,0,3,0
299,45,1,3,110,264,0,1,132,0,1.2,1,0,3,0
300,68,1,0,144,193,1,1,141,0,3.4,1,2,3,0
301,57,1,0,130,131,0,1,115,1,1.2,1,1,3,0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303 entries, 0 to 302
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       303 non-null    int64  
 1   sex       303 non-null    int64  
 2   cp        303 non-null    int64  
 3   trestbps  303 non-null    int64  
 4   chol      303 non-null    int64  
 5   fbs       303 non-null    int64  
 6   restecg   303 non-null    int64  
 7   thalach   303 non-null    int64  
 8   exang     303 non-null    int64  
 9   oldpeak   303 non-null    float64
 10  slope     303 non-null    int64  
 11  ca        303 non-null    int64  
 12  thal      303 non-null    int64  
 13  target    303 non-null    int64  
dtypes: float64(1), int64(13)
memory usage: 33.3 KB


In [5]:
df.describe()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
count,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000
mean,54.366337,0.683168,0.966997,131.623762,246.264026,0.148515,0.528053,149.646865,0.326733,1.039604,1.399340,0.729373,2.313531,0.544554
std,9.082101,0.466011,1.032052,17.538143,51.830751,0.356198,0.525860,22.905161,0.469794,1.161075,0.616226,1.022606,0.612277,0.498835
min,29.000000,0.000000,0.000000,94.000000,126.000000,0.000000,0.000000,71.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,47.500000,0.000000,0.000000,120.000000,211.000000,0.000000,0.000000,133.500000,0.000000,0.000000,1.000000,0.000000,2.000000,0.000000
50%,55.000000,1.000000,1.000000,130.000000,240.000000,0.000000,1.000000,153.000000,0.000000,0.800000,1.000000,0.000000,2.000000,1.000000
75%,61.000000,1.000000,2.000000,140.000000,274.500000,0.000000,1.000000,166.000000,1.000000,1.600000,2.000000,1.000000,3.000000,1.000000
max,77.000000,1.000000,3.000000,200.000000,564.000000,1.000000,2.000000,202.000000,1.000000,6.200000,2.000000,4.000000,3.000000,1.000000


In [6]:
df.nunique()

age          41
sex           2
cp            4
trestbps     49
chol        152
fbs           2
restecg       3
thalach      91
exang         2
oldpeak      40
slope         3
ca            5
thal          4
target        2
dtype: int64

Видим что признаки **age**, **trestbps**, **chol**, **thalach**, **oldpeak** являются непрерывными, а остальные-категориальные

напишем пайплайн для обработки признаков

In [7]:
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.base import BaseEstimator, TransformerMixin

class ColumnSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.key]
    
class NumberSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    Use on numeric columns in the data
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[[self.key]]
    
class OHEEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
        self.columns = []

    def fit(self, X, y=None):
        self.columns = [col for col in pd.get_dummies(X, prefix=self.key).columns]
        return self

    def transform(self, X):
        X = pd.get_dummies(X, prefix=self.key)
        test_columns = [col for col in X.columns]
        for col_ in test_columns:
            if col_ not in self.columns:
                X[col_] = 0
        return X[self.columns]


from sklearn.preprocessing import StandardScaler


continuos_cols = ['age', 'trestbps', 'chol', 'thalach', 'oldpeak']
cat_cols = ['sex','cp', 'fbs', 'exang', 'slope']
base_cols = ['restecg', 'ca', 'thal']


continuos_transformers = []
cat_transformers = []
base_transformers=[]


for cont_col in continuos_cols:
    transfomer =  Pipeline([
                ('selector', NumberSelector(key=cont_col)),
                ('standard', StandardScaler())
            ])
    continuos_transformers.append((cont_col, transfomer))
    
for cat_col in cat_cols:
    cat_transformer = Pipeline([
                ('selector', ColumnSelector(key=cat_col)),
                ('ohe', OHEEncoder(key=cat_col))
            ])
    cat_transformers.append((cat_col, cat_transformer))
    
for base_col in base_cols:
    base_transformer = Pipeline([
                ('selector', NumberSelector(key=base_col))
            ])
    base_transformers.append((base_col, base_transformer))

In [8]:
from sklearn.model_selection import train_test_split
#разделим данные на train/test
X_train, X_test, y_train, y_test = train_test_split(df.drop('target', 1), 
                                                    df['target'], random_state=24)

C:\Users\Marat\AppData\Local\Temp\ipykernel_260\4128685706.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  X_train, X_test, y_train, y_test = train_test_split(df.drop('target', 1),


In [9]:
from sklearn.pipeline import FeatureUnion

feats = FeatureUnion(continuos_transformers+cat_transformers+base_transformers)
feature_processing = Pipeline([('feats', feats)])

feature_processing.fit_transform(X_train)

array([[-6.75941412e-01, -4.34565489e-01,  5.14025007e-01, ...,
         0.00000000e+00,  0.00000000e+00,  3.00000000e+00],
       [ 5.49785317e-01,  1.37629209e-01, -8.13865964e-01, ...,
         1.00000000e+00,  2.00000000e+00,  2.00000000e+00],
       [-1.23308993e+00,  2.31902697e-02,  1.83848970e-03, ...,
         0.00000000e+00,  4.00000000e+00,  3.00000000e+00],
       ...,
       [ 2.22123086e+00, -6.63443368e-01,  4.19175652e-01, ...,
         0.00000000e+00,  1.00000000e+00,  2.00000000e+00],
       [ 1.77551205e+00,  1.39645754e+00, -3.61012523e-02, ...,
         0.00000000e+00,  0.00000000e+00,  2.00000000e+00],
       [-7.36319621e-03, -6.63443368e-01, -1.11738390e+00, ...,
         1.00000000e+00,  1.00000000e+00,  3.00000000e+00]])

In [10]:
from sklearn.ensemble import RandomForestClassifier
cls = Pipeline([
    ('features',feats),
    ('classifier', RandomForestClassifier(random_state = 0)),
])
cls.fit(X_train, y_train)


Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('age',
                                                 Pipeline(steps=[('selector',
                                                                  NumberSelector(key='age')),
                                                                 ('standard',
                                                                  StandardScaler())])),
                                                ('trestbps',
                                                 Pipeline(steps=[('selector',
                                                                  NumberSelector(key='trestbps')),
                                                                 ('standard',
                                                                  StandardScaler())])),
                                                ('chol',
                                                 Pipeline(steps=[('selector',
                                      

In [11]:
y_pred=cls.predict(X_test)

Проверим качество

In [12]:
results = {
    'approach': [],
    'f1': [],
    'roc': [],
    'recall':[],
    'precision':[]
}

from sklearn.metrics import recall_score, precision_score, roc_auc_score, accuracy_score, f1_score

def evaluate_results(y_test, y_predict):
    print('Classification results:')
    f1 = f1_score(y_test, y_predict)
    print("f1: %.2f%%" % (f1 * 100.0)) 
    roc = roc_auc_score(y_test, y_predict)
    print("roc: %.2f%%" % (roc * 100.0)) 
    rec = recall_score(y_test, y_predict, average='binary')
    print("recall: %.2f%%" % (rec * 100.0)) 
    prc = precision_score(y_test, y_predict, average='binary')
    print("precision: %.2f%%" % (prc * 100.0)) 



results['approach'].append('rfc')
results['f1'].append(f1_score(y_test, y_pred))
results['roc'].append(roc_auc_score(y_test, y_pred))
results['recall'].append(recall_score(y_test, y_pred, average='binary'))
results['precision'].append(precision_score(y_test, y_pred, average='binary'))

evaluate_results(y_test, y_pred)

Classification results:
f1: 84.34%
roc: 82.68%
recall: 85.37%
precision: 83.33%


### PU learning

In [13]:
mod_data = df.copy()
#get the indices of the positives samples
pos_ind = np.where(mod_data.iloc[:,-1].values == 1)[0]
#shuffle them
np.random.shuffle(pos_ind)
# leave just 25% of the positives marked
pos_sample_len = int(np.ceil(0.25 * len(pos_ind)))
print(f'Using {pos_sample_len}/{len(pos_ind)} as positives and unlabeling the rest')
pos_sample = pos_ind[:pos_sample_len]

Using 42/165 as positives and unlabeling the rest


Создаем столбец для новой целевой переменной, где у нас два класса - P (1) и U (-1)

In [14]:
mod_data['class_test'] = -1
mod_data.loc[pos_sample,'class_test'] = 1
print('target variable:\n', mod_data.iloc[:,-1].value_counts())

target variable:
 -1    261
 1     42
Name: class_test, dtype: int64


In [15]:
x_data = mod_data.iloc[:,:-2].values # just the X 
y_labeled = mod_data.iloc[:,-1].values # new class (just the P & U)
y_positive = mod_data.iloc[:,-2].values # original class

### 1. random negative sampling

In [16]:
mod_data = mod_data.sample(frac=1)
neg_sample = mod_data[mod_data['class_test']==-1][:len(mod_data[mod_data['class_test']==1])]
sample_test = mod_data[mod_data['class_test']==-1][len(mod_data[mod_data['class_test']==1]):]
pos_sample = mod_data[mod_data['class_test']==1]
print(neg_sample.shape, pos_sample.shape)
sample_train = pd.concat([neg_sample, pos_sample]).sample(frac=1)

(42, 15) (42, 15)


In [17]:
cls1=RandomForestClassifier(random_state = 0)
cls1.fit(sample_train.iloc[:,:-2].values, 
          sample_train.iloc[:,-2].values)
y_predict = cls1.predict(sample_test.iloc[:,:-2].values)

results['approach'].append('rfc_25%_rns')
results['f1'].append(f1_score(sample_test.iloc[:,-2].values, y_predict))
results['roc'].append(roc_auc_score(sample_test.iloc[:,-2].values, y_predict))
results['recall'].append(recall_score(sample_test.iloc[:,-2].values, y_predict, average='binary'))
results['precision'].append(precision_score(sample_test.iloc[:,-2].values, y_predict, average='binary'))


evaluate_results(sample_test.iloc[:,-2].values, y_predict)

Classification results:
f1: 74.71%
roc: 72.95%
recall: 97.96%
precision: 60.38%


### 2. random negative sampling 15%

In [18]:
mod_data = df.copy()
#get the indices of the positives samples
pos_ind = np.where(mod_data.iloc[:,-1].values == 1)[0]
#shuffle them
np.random.shuffle(pos_ind)
# leave just 25% of the positives marked
pos_sample_len = int(np.ceil(0.15 * len(pos_ind)))
print(f'Using {pos_sample_len}/{len(pos_ind)} as positives and unlabeling the rest')
pos_sample = pos_ind[:pos_sample_len]

Using 25/165 as positives and unlabeling the rest


In [19]:
mod_data['class_test'] = -1
mod_data.loc[pos_sample,'class_test'] = 1
print('target variable:\n', mod_data.iloc[:,-1].value_counts())

target variable:
 -1    278
 1     25
Name: class_test, dtype: int64


In [20]:
x_data = mod_data.iloc[:,:-2].values # just the X 
y_labeled = mod_data.iloc[:,-1].values # new class (just the P & U)
y_positive = mod_data.iloc[:,-2].values # original class

In [21]:
mod_data = mod_data.sample(frac=1)
neg_sample = mod_data[mod_data['class_test']==-1][:len(mod_data[mod_data['class_test']==1])]
sample_test = mod_data[mod_data['class_test']==-1][len(mod_data[mod_data['class_test']==1]):]
pos_sample = mod_data[mod_data['class_test']==1]
print(neg_sample.shape, pos_sample.shape)
sample_train = pd.concat([neg_sample, pos_sample]).sample(frac=1)

(25, 15) (25, 15)


In [23]:
cls2=RandomForestClassifier(random_state = 0)
cls2.fit(sample_train.iloc[:,:-2].values, 
          sample_train.iloc[:,-2].values)
y_predict = cls2.predict(sample_test.iloc[:,:-2].values)

results['approach'].append('rfc_15%_rns')
results['f1'].append(f1_score(sample_test.iloc[:,-2].values, y_predict))
results['roc'].append(roc_auc_score(sample_test.iloc[:,-2].values, y_predict))
results['recall'].append(recall_score(sample_test.iloc[:,-2].values, y_predict, average='binary'))
results['precision'].append(precision_score(sample_test.iloc[:,-2].values, y_predict, average='binary'))


evaluate_results(sample_test.iloc[:,-2].values, y_predict)

Classification results:
f1: 78.48%
roc: 72.84%
recall: 96.88%
precision: 65.96%


### 3. random negative sampling 50%

In [24]:
mod_data = df.copy()
#get the indices of the positives samples
pos_ind = np.where(mod_data.iloc[:,-1].values == 1)[0]
#shuffle them
np.random.shuffle(pos_ind)
# leave just 25% of the positives marked
pos_sample_len = int(np.ceil(0.5 * len(pos_ind)))
print(f'Using {pos_sample_len}/{len(pos_ind)} as positives and unlabeling the rest')
pos_sample = pos_ind[:pos_sample_len]

Using 83/165 as positives and unlabeling the rest


In [25]:
mod_data['class_test'] = -1
mod_data.loc[pos_sample,'class_test'] = 1
print('target variable:\n', mod_data.iloc[:,-1].value_counts())

target variable:
 -1    220
 1     83
Name: class_test, dtype: int64


In [26]:
x_data = mod_data.iloc[:,:-2].values # just the X 
y_labeled = mod_data.iloc[:,-1].values # new class (just the P & U)
y_positive = mod_data.iloc[:,-2].values # original class

In [27]:
mod_data = mod_data.sample(frac=1)
neg_sample = mod_data[mod_data['class_test']==-1][:len(mod_data[mod_data['class_test']==1])]
sample_test = mod_data[mod_data['class_test']==-1][len(mod_data[mod_data['class_test']==1]):]
pos_sample = mod_data[mod_data['class_test']==1]
print(neg_sample.shape, pos_sample.shape)
sample_train = pd.concat([neg_sample, pos_sample]).sample(frac=1)

(83, 15) (83, 15)


In [28]:
cls3=RandomForestClassifier(random_state = 0)
cls3.fit(sample_train.iloc[:,:-2].values, 
          sample_train.iloc[:,-2].values)
y_predict = cls3.predict(sample_test.iloc[:,:-2].values)

results['approach'].append('rfc_50%_rns')
results['f1'].append(f1_score(sample_test.iloc[:,-2].values, y_predict))
results['roc'].append(roc_auc_score(sample_test.iloc[:,-2].values, y_predict))
results['recall'].append(recall_score(sample_test.iloc[:,-2].values, y_predict, average='binary'))
results['precision'].append(precision_score(sample_test.iloc[:,-2].values, y_predict, average='binary'))


evaluate_results(sample_test.iloc[:,-2].values, y_predict)

Classification results:
f1: 82.17%
roc: 84.98%
recall: 94.64%
precision: 72.60%


In [29]:
pd.DataFrame(data=results).sort_values(by='f1', ascending=False)

,approach,f1,roc,recall,precision
0,rfc,0.843373,0.826829,0.853659,0.833333
4,rfc_50%_rns,0.821705,0.849757,0.946429,0.726027
2,rfc_15%_rns,0.784810,0.728375,0.968750,0.659574
3,rfc_15%_rns,0.784810,0.728375,0.968750,0.659574
1,rfc_25%_rns,0.747082,0.729465,0.979592,0.603774
